In [1]:
import numpy as np
import random



In [2]:
# Grid initialization
grid_size = 100
grid = np.zeros((grid_size, grid_size))


In [3]:
# Obstacles placement
num_obstacles = 2000
obstacle_positions = random.sample(range(grid_size * grid_size), num_obstacles)
for pos in obstacle_positions:
    grid[pos // grid_size, pos % grid_size] = -1


In [4]:
# Start and goal points
start = (random.randint(0, grid_size-1), random.randint(0, grid_size-1))
goal = (random.randint(0, grid_size-1), random.randint(0, grid_size-1))


In [5]:
# MDP components
actions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
rewards = np.zeros((grid_size, grid_size))
rewards[goal] = 100
rewards[grid == -1] = -100


In [6]:
# Value Iteration
def value_iteration(grid, rewards, actions, gamma=0.9, theta=1e-4):
    V = np.zeros((grid_size, grid_size))
    while True:
        delta = 0
        for i in range(grid_size):
            for j in range(grid_size):
                v = V[i, j]
                action_values = []
                for a in actions:
                    ni, nj = i + a[0], j + a[1]
                    if 0 <= ni < grid_size and 0 <= nj < grid_size:
                        action_values.append(rewards[ni, nj] + gamma * V[ni, nj])
                    else:
                        action_values.append(-np.inf)  # Penalize invalid moves
                V[i, j] = max(action_values)
                delta = max(delta, abs(v - V[i, j]))
        if delta < theta:
            break
    policy = np.zeros((grid_size, grid_size), dtype=int)
    for i in range(grid_size):
        for j in range(grid_size):
            action_values = []
            for a in actions:
                ni, nj = i + a[0], j + a[1]
                if 0 <= ni < grid_size and 0 <= nj < grid_size:
                    action_values.append(rewards[ni, nj] + gamma * V[ni, nj])
                else:
                    action_values.append(-np.inf)  # Penalize invalid moves
            policy[i, j] = np.argmax(action_values)
    return V, policy

In [7]:
# Q-Learning
def q_learning(grid, rewards, actions, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=1000):
    Q = np.zeros((grid_size, grid_size, len(actions)))
    for episode in range(episodes):
        state = start
        while state != goal:
            if random.uniform(0, 1) < epsilon:
                action = random.choice(range(len(actions)))
            else:
                action = np.argmax(Q[state[0], state[1]])
            next_state = (state[0] + actions[action][0], state[1] + actions[action][1])
            if 0 <= next_state[0] < grid_size and 0 <= next_state[1] < grid_size:
                reward = rewards[next_state[0], next_state[1]]
                Q[state[0], state[1], action] += alpha * (reward + gamma * np.max(Q[next_state[0], next_state[1]]) - Q[state[0], state[1], action])
                state = next_state
            else:
                break  # Stop if the next state is out of bounds
    policy = np.argmax(Q, axis=2)
    return Q, policy

In [8]:
V_vi, policy_vi = value_iteration(grid, rewards, actions)
Q_ql, policy_ql = q_learning(grid, rewards, actions)


In [9]:
import time
import matplotlib.pyplot as plt


In [10]:
# Function to simulate the path taken by a policy
def simulate_path(grid, policy, start, goal):
    state = start
    path = [state]
    steps = 0
    while state != goal:
        action = policy[state[0], state[1]]
        next_state = (state[0] + actions[action][0], state[1] + actions[action][1])
        if 0 <= next_state[0] < grid_size and 0 <= next_state[1] < grid_size:
            state = next_state
            path.append(state)
            steps += 1
        else:
            break  # Stop if the next state is out of bounds
    return path, steps


In [ ]:
# Simulate paths for each method
path_vi, steps_vi = simulate_path(grid, policy_vi, start, goal)
path_ql, steps_ql = simulate_path(grid, policy_ql, start, goal)


In [ ]:
# Visualize paths
def plot_path(grid, path, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='gray')
    path_x, path_y = zip(*path)
    plt.plot(path_y, path_x, marker='o', markersize=3, color='red')
    plt.scatter(start[1], start[0], marker='*', color='green', s=200)
    plt.scatter(goal[1], goal[0], marker='*', color='blue', s=200)
    plt.title(title)
    plt.show()


In [ ]:
plot_path(grid, path_vi, 'Value Iteration Path')
plot_path(grid, path_ql, 'Q-Learning Path')


In [ ]:
print(f"Value Iteration Steps: {steps_vi}")
print(f"Q-Learning Steps: {steps_ql}")
